In [ ]:
#| label: irFig3jn

from repo2data.repo2data import Repo2Data
import os 
import pickle
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
from plotly import tools
from pathlib import Path

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(os.path.abspath('')).parents[2]
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)
    
DATA_ROOT = dataset_path / "t1-book-neurolibre"
filename = os.path.join(DATA_ROOT,"01",'figure_2.pkl')

with open(filename, 'rb') as f:
    params, signal_WM, signal_GM, signal_CSF = pickle.load(f)

filename = os.path.join(DATA_ROOT,"01",'figure_3.pkl')
with open(filename, 'rb') as f:
    T1_range, signal_T1_Eq1, signal_T1_Eq3 = pickle.load(f)

config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        x = params["TI"],
        y = abs(signal_T1_Eq3[ii]),
        name = '[Eq. 2.3] – Long TR approximation',
        text = '[Eq. 2.3] – Long TR approximation',
        hoverinfo = 'x+y+text') for ii in range(len(T1_range))]

data1[3]['visible'] = True

data2 = [dict(
        visible = False,
        x = params["TI"],
        y = abs(signal_T1_Eq1[ii]),
        line = dict(
            color = ('rgb(22, 96, 167)'),
            dash = 'dash'),
        name = '[Eq. 2.1] – General Equation',
        text = '[Eq. 2.1] – General Equation',
        hoverinfo = 'x+y+text') for ii in range(len(T1_range))]

data2[3]['visible'] = True

data = data1 + data2

steps = []
for i in range(len(T1_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(T1_range[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 3,
    currentvalue = {"prefix": "T<sub>1</sub> value (s): <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=580,
    height=400,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.2,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.14,
            y=0.5,
            showarrow=False,
            text='Signal (magnitude)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, 5],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.5,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders,
    plot_bgcolor='white'
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'ir_fig_3.html', config = config)
